## Unstructed比较各种模式 解析PDF文件

In [2]:
# 初始化 PDF文件路径
# file_path = "../../90-文档_Data/山西文旅/云冈石窟-ch.pdf"
file_path = "../../90-文档_Data/test/xiaomi14.pdf"

#### 1. 使用LangChain的Unstrucured PDF解析

In [6]:
from langchain_unstructured import UnstructuredLoader
import pprint

loader = UnstructuredLoader(
    file_path=file_path,  # PDF文件路径
    strategy="hi_res",  # 使用高分辨率策略进行文档处理
    # partition_via_api=True,  # 通过API进行文档分块
    # coordinates=True,     # 提取文本坐标信息
)
docs = []
# lazy_load() 是一种延迟加载方法
# 它不会一次性将所有文档加载到内存中，而是在需要时才逐个加载文档
# 这对于处理大型PDF文件时可以节省内存使用
for doc in loader.lazy_load():
    docs.append(doc)

# 打印每个文档的元数据
for doc in docs[:5]:
    pprint.pprint(doc.model_dump())

INFO: Reading PDF for file: ../../90-文档_Data/test/xiaomi14.pdf ...


{'id': None,
 'metadata': {'category': 'NarrativeText',
              'coordinates': {'layout_height': 2339,
                              'layout_width': 1654,
                              'points': ((np.float64(241.27891540527344),
                                          np.float64(285.5626220703125)),
                                         (np.float64(241.27891540527344),
                                          np.float64(841.6060180664062)),
                                         (np.float64(1404.8731689453125),
                                          np.float64(841.6060180664062)),
                                         (np.float64(1404.8731689453125),
                                          np.float64(285.5626220703125))),
                              'system': 'PixelSpace'},
              'detection_class_prob': 0.9352302551269531,
              'element_id': '75c3feeba185e3ba5dddbb2e4ddee6fa',
              'file_directory': '../../90-文档_Data/test',
            

#### 2. 使用LangChain的Unstrucured PDF解析 提取文档结构

In [7]:
from langchain_unstructured import UnstructuredLoader

loader = UnstructuredLoader(
    file_path=file_path,
    strategy="hi_res",
    # partition_via_api=True,
    # coordinates=True,
)
docs = []
for doc in loader.lazy_load():
    docs.append(doc)


def extract_basic_structure(docs):
    """基础结构化提取:按文档类型组织内容"""
    # 定义类别映射
    category_map = {
        "Title": "title",
        "NarrativeText": "text",
        "Image": "image",
        "Table": "table",
        "Footer": "footer",
        "Header": "header",
    }

    # 初始化结构字典
    structure = {cat: [] for cat in category_map.values()}
    structure["metadata"] = []  # 额外添加metadata类别

    # 遍历文档并分类
    for doc in docs:
        category = doc.metadata.get("category", "Unknown")
        content = {
            "content": doc.page_content,
            "page": doc.metadata.get("page_number"),
            "coordinates": doc.metadata.get("coordinates"),
        }

        target_category = category_map.get(category)
        if target_category:
            structure[target_category].append(content)

    return structure


# 使用示例
structure = extract_basic_structure(docs)

# 输出第2页的标题
print("第2页标题:")
for title in [t for t in structure["title"] if t["page"] == 2]:
    print(f"- {title['content']}")


def analyze_layout(docs):
    """分析文档的版面布局结构"""
    layout_analysis = {}

    for doc in docs:
        page = doc.metadata.get("page_number")
        coords = doc.metadata.get("coordinates", {})

        # 初始化页面信息
        if page not in layout_analysis:
            layout_analysis[page] = {
                "elements": [],
                "dimensions": {
                    "width": coords.get("layout_width", 0),
                    "height": coords.get("layout_height", 0),
                },
            }

        # 获取元素位置信息
        points = coords.get("points", [])
        if points:
            # 只需要左上和右下坐标点
            (x1, y1), (_, _), (x2, y2), _ = points

            # 构建元素信息
            element = {
                "type": doc.metadata.get("category"),
                "content": (
                    (doc.page_content[:50] + "...")
                    if len(doc.page_content) > 50
                    else doc.page_content
                ),
                "position": {
                    "x1": x1,
                    "y1": y1,
                    "x2": x2,
                    "y2": y2,
                    "width": x2 - x1,
                    "height": y2 - y1,
                },
            }
            layout_analysis[page]["elements"].append(element)

    return layout_analysis


# 使用示例
layout = analyze_layout(docs)

# 分析第一页的布局
print("第1页布局分析:")
if 1 in layout:
    page = layout[1]
    print(f"页面尺寸: {page['dimensions']['width']} x {page['dimensions']['height']}")
    print("\n元素分布:")

    # 按垂直位置排序显示元素
    for elem in sorted(page["elements"], key=lambda x: x["position"]["y1"]):
        print(f"\n类型: {elem['type']}")
        print(f"位置: ({elem['position']['x1']:.0f}, {elem['position']['y1']:.0f})")
        print(
            f"尺寸: {elem['position']['width']:.0f} x {elem['position']['height']:.0f}"
        )
        print(f"内容: {elem['content']}")

# 寻找父子关系
cave6_docs = []
parent_id = -1
for doc in docs:
    if doc.metadata["category"] == "Title" and "Cave 6" in doc.page_content:
        parent_id = doc.metadata["element_id"]
    if doc.metadata.get("parent_id") == parent_id:
        cave6_docs.append(doc)

for doc in cave6_docs:
    print(doc.page_content)

external_docs = []  # 创建列表来存储外部链接的子文档
parent_id = -1  # 初始化父ID为-1
for doc in docs:
    # 检查文档是否为标题类型且内容包含"External links"
    if doc.metadata["category"] == "Title" and "External links" in doc.page_content:
        parent_id = doc.metadata["element_id"]
        external_docs.append(doc)
    # 检查文档的parent_id是否匹配我们找到的标题ID
    if doc.metadata.get("parent_id") == parent_id:
        external_docs.append(doc)  # 将属于这个标题的子文档都添加到结果列表中
for doc in external_docs:
    print(doc.page_content)


# def find_tables_and_titles(docs):
#   results = []
#   for doc in docs:
#     # 检查文档是否为表格类型
#     if doc.metadata.get("category") == "Table":
#       table = doc
#       parent_id = doc.metadata.get("parent_id")
#       # 查找表格对应的标题文档(parent_id匹配element_id)
#       title = next((doc for doc in docs if doc.metadata.get("element_id") == parent_id), None)
#       if title:
#         results.append({"table": table.page_content, "title": title.page_content})
#   return results

# results = find_tables_and_titles(cave6_docs)
# if results:
#   for result in results:
#     print("找到的表格和标题:")
#     print(f"标题: {result['title']}")
#     print(f"表格: {result['table']}")
# else:
#   print("未找到任何表格和标题")

INFO: Reading PDF for file: ../../90-文档_Data/test/xiaomi14.pdf ...


第2页标题:
- 富的细节。
- 片。
第1页布局分析:
页面尺寸: 1654 x 2339

元素分布:

类型: NarrativeText
位置: (241, 286)
尺寸: 1164 x 556
内容: 随着科技的飞速发展，手机摄影早已不再是简单的记 录，而是成为展现个性、捕捉美好瞬间的艺术工具。 在众...

类型: Title
位置: (250, 814)
尺寸: 350 x 50
内容: 动影像的标杆。

类型: Image
位置: (250, 947)
尺寸: 1153 x 926
内容: Bib se RIRANM ER XIBOMI 14 Ultra X13gOMI 13 Ultra ...

类型: Title
位置: (250, 1981)
尺寸: 861 x 63
内容: 一、影像系统：专业级配置，画质超凡


#### 3. 原生Unstructed 使用partition函数解析PDF

In [8]:
# 导入unstructured的partition函数用于PDF解析
from unstructured.partition.auto import partition

# 使用partition函数解析PDF文件
# content_type指定文件类型为PDF
elements = partition(
    filename=file_path, 
    content_type="application/pdf"
    )

# 展示解析出的elements的类型和内容
print("PDF解析后的Elements类型:")
for i, element in enumerate(elements[:5]):
    print(f"\nElement {i+1}:")
    print(f"类型: {type(element).__name__}")
    print(f"内容: {str(element)}")
    print("-" * 50)

# 统计不同类型elements的数量
element_types = {}
for element in elements:
    element_type = type(element).__name__
    element_types[element_type] = element_types.get(element_type, 0) + 1

print("\nElements类型统计:")
for element_type, count in element_types.items():
    print(f"{element_type}: {count}个")

PDF解析后的Elements类型:

Element 1:
类型: Title
内容: 随着科技的飞速发展，手机摄影早已不再是简单的记
--------------------------------------------------

Element 2:
类型: Title
内容: 录，而是成为展现个性、捕捉美好瞬间的艺术工具。
--------------------------------------------------

Element 3:
类型: Title
内容: 在众多影像旗舰中，小米 14 Ultra 以其卓越的影像实
--------------------------------------------------

Element 4:
类型: Title
内容: 力和专业级的拍摄体验，成为了摄影爱好者及普通用
--------------------------------------------------

Element 5:
类型: Title
内容: 户的新宠。今天，就让我们一起深度体验这款被誉为
--------------------------------------------------

Elements类型统计:
Title: 64个
NarrativeText: 2个
Text: 1个


#### 4. 原生Unstructed 使用 partition_pdf 函数解析PDF

In [3]:
from unstructured.partition.pdf import partition_pdf

# 解析PDF
# elements = partition_pdf(filename=file_path)

elements = partition_pdf(
    filename=file_path,
    strategy="hi_res",  # 使用高精度模式，普通模式是fast
    chunking_strategy="by_title",  # 按标题分块, 普通模式是Basic
    # max_characters=2500,  # 每个分块的最大字符数
    # new_after_n_chars=2300,  # 在达到指定字符数后开始新分块
    infer_table_structure=True,  # 推断表格结构
    extract_images=True,  # 提取图像
    image_format="png",  # 图像格式
    include_metadata=True,  # 包含元数据
)

# 展示解析后的内容
for i, element in enumerate(elements[:10]):  # 只展示前10个元素
    print(f"Element {i+1}:")
    print(element)
    print("-" * 80)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Element 1:
随着科技的飞速发展，手机摄影早已不再是简单的记 录，而是成为展现个性、捕捉美好瞬间的艺术工具。 在众多影像旗舰中，小米 14 Ultra 以其卓越的影像实 力和专业级的拍摄体验，成为了摄影爱好者及普通用 户的新宠。今天，就让我们一起深度体验这款被誉为 “能打电话的相机”的小米 14 Ultra，探索它如何重塑移

动影像的标杆。

Bib se RIRANM ER XIBOMI 14 Ultra X13gOMI 13 Ultra RE e SSRI

一、影像系统：专业级配置，画质超凡

小米 14 Ultra 在影像系统上可谓下足了功夫。它搭载 了一英寸大底传感器，配合索尼 LYT-900 传感器，带 来更高的感光能力和更宽广的动态范围。无论是白天 还是夜晚，小米 14 Ultra 都能捕捉到细腻的画面和丰
--------------------------------------------------------------------------------
Element 2:
富的细节。

更令人惊喜的是，小米 14 Ultra 的主摄采用了 F1.63 超大光圈，并支持 F1.63～F4.0 无极可变光圈设计。 这一设计使得手机主摄可以在不同场景下灵活调整光 圈大小，实现更出色的背景虚化效果，让主体更加突 出。无论是拍摄人像还是风景，都能轻松拍出大片感。

此外，小米 14 Ultra 还配备了超广角镜头、75mm 长 焦镜头和 120mm 潜望式长焦镜头，覆盖了从广角到 长焦的多个焦段。这使得用户在不同场景下都能找到 合适的拍摄视角，轻松拍出具有层次感和空间感的照
--------------------------------------------------------------------------------
Element 3:
片。

12GB+256GB ¥ 6499 12GB+256GB ¥ 5399 ne 16GB+512GB ¥6999 16GB+512GB ¥ 5899 16GB+1TB ¥7799 16GB+1TB ¥ 6699 16GB+1TB ¥ 8799 S=KRE 8 BMFG BI-KRE °8 BMA EAE LPDDR5X + UFS 4.0 LPDDR5X + UFS 4.0 + 